<a href="https://colab.research.google.com/github/MarcoMinozzo/Marketplace_API/blob/main/Amazon_SP_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

To obtain the variables listed in the table using **Amazon’s Selling Partner API (SP-API)**, here is a detailed overview of what data is directly available, the endpoints required, and alternative methods for data not directly accessible through the API.

### 1. **Directly Available Variables in SP-API**

| Data Category               | Variable Name              | Endpoint and Method                               |
|-----------------------------|----------------------------|---------------------------------------------------|
| **Customer Feedback**       | `feedback_rating`          | `GET /orders/v0/orders`                           |
| **Customer Feedback**       | `feedback_comments`        | `GET /orders/v0/orders`                           |
| **Customer Service Metrics**| `order_defect_rate`        | `GET /orders/v0/orders` or `GET /reports/v0/reports`|
| **Customer Service Metrics**| `customer_service_metrics` | `GET /reports/v0/reports` (custom reports)        |
| **Return Reasons**          | `return_reasons`           | `GET /returns/v0/returns`                         |
| **Return Reasons**          | `buyer_messages`           | `GET /messaging/v1/messages`                      |

### Explanation of Variables and How to Collect Them

1. **Order Feedback (feedback_rating and feedback_comments)**:
   - **Endpoint**: `GET /orders/v0/orders`
   - **Description**: Customer feedback on orders is accessible through the Orders endpoint, allowing you to see general feedback through fields like `OrderStatus` and `buyerInfo`.
   - **Example Parameter**: `{"OrderStatuses": ["Shipped"], "CreatedAfter": "2023-01-01T00:00:00Z"}` for filtered, specific orders.

2. **Customer Service Metrics (order_defect_rate and customer_service_metrics)**:
   - **Endpoint**: `GET /reports/v0/reports`
   - **Description**: Metrics like order defect rate (`order_defect_rate`) can be accessed through custom reports. The reports endpoint offers the option to retrieve performance metrics, including customer service data.
   - **Example Configuration**: A report with specific customer service metrics can be requested using the report type **"GET_FLAT_FILE_ALL_ORDERS_DATA_BY_LAST_UPDATE_GENERAL"**.

3. **Return Reasons and Buyer Messages (return_reasons and buyer_messages)**:
   - **Endpoints**: `GET /returns/v0/returns` for returns data and `GET /messaging/v1/messages` for buyer communications.
   - **Description**: To access return data, including reasons (`return_reasons`), use the Returns endpoint. Buyer messages, such as feedback or inquiries, are available through the Messaging endpoint, helping identify complaints or frequent issues.

### 2. **Variables Not Directly Accessible (Alternative Approaches)**

Unfortunately, **Product Reviews** variables (`rating`, `review_text`, `review_title`, and `helpful_votes`) are not directly accessible through SP-API due to privacy policies. Here are alternatives:

- **Alternative for Product Reviews**: Use third-party services or **review analytics APIs**, like **ReviewMeta** or **Fakespot**, which provide general insights into product ratings. Alternatively, social media or public review analysis platforms can supplement the information.

### Code for Available Variables

Below is a sample code snippet to retrieve **Order Feedback** and **Buyer Messages**:


### Summary

- **Order Feedback and Buyer Messages**: Available directly in SP-API.
- **Product Reviews**: Not directly accessible; consider using third-party review analysis services.
- **Returns and Service Metrics**: Accessible via specific SP-API endpoints.


In [ ]:
import requests
from datetime import datetime
from botocore.auth import SigV4Auth
from botocore.awsrequest import AWSRequest
import boto3

# SP-API Authentication
AWS_ACCESS_KEY = 'YOUR_AWS_ACCESS_KEY'
AWS_SECRET_KEY = 'YOUR_AWS_SECRET_KEY'
AWS_REGION = 'us-east-1'
ACCESS_TOKEN = 'YOUR_LWA_ACCESS_TOKEN'

# Order Feedback
def fetch_order_feedback():
    endpoint = "https://sellingpartnerapi-na.amazon.com"
    path = "/orders/v0/orders"
    url = f"{endpoint}{path}"
    params = {
        'MarketplaceIds': 'ATVPDKIKX0DER',
        'CreatedAfter': '2023-01-01T00:00:00Z',
    }
    headers = {
        'x-amz-access-token': ACCESS_TOKEN,
        'x-amz-date': datetime.utcnow().strftime('%Y%m%dT%H%M%SZ'),
        'Host': 'sellingpartnerapi-na.amazon.com'
    }
    request = AWSRequest(method="GET", url=url, params=params, headers=headers)
    SigV4Auth(boto3.Session().get_credentials(), "execute-api", AWS_REGION).add_auth(request)
    response = requests.get(url, headers=dict(request.headers), params=params)
    if response.status_code == 200:
        return response.json()
    else:
        print("Error:", response.status_code, response.text)
        return None

# Buyer Messages
def fetch_buyer_messages():
    endpoint = "https://sellingpartnerapi-na.amazon.com"
    path = "/messaging/v1/messages"
    url = f"{endpoint}{path}"
    headers = {
        'x-amz-access-token': ACCESS_TOKEN,
        'x-amz-date': datetime.utcnow().strftime('%Y%m%dT%H%M%SZ'),
        'Host': 'sellingpartnerapi-na.amazon.com'
    }
    request = AWSRequest(method="GET", url=url, headers=headers)
    SigV4Auth(boto3.Session().get_credentials(), "execute-api", AWS_REGION).add_auth(request)
    response = requests.get(url, headers=dict(request.headers))
    if response.status_code == 200:
        return response.json()
    else:
        print("Error:", response.status_code, response.text)
        return None

# Run functions
order_feedback = fetch_order_feedback()
buyer_messages = fetch_buyer_messages()
print(order_feedback)
print(buyer_messages)